In [ ]:
#some functions and libraries

in moodle mentioned:
    for classifiers:
        naive bayes # https://scikit-learn.org/stable/modules/naive_bayes.html
        decision tree
        knn
        suport vector machines
        logistic regression
        ensemble methods
    for evaluation: (all used in examples here)
        accuracy
        cross validation
        confusion matrix
        grid search
    for text analysis:
        feature extraction # https://scikit-learn.org/stable/modules/classes.html#text-feature-extraction-ref
        NLTK # https://www.nltk.org/
    some visualization
    

In [ ]:
## confusion matrix, dummy encoding, knn

In [ ]:
dataset.isna().any() #Ist der Datensatz vollständig (d. h. gibt es keine Nullwerte)? missing values

In [ ]:
df.hist()

In [ ]:
X=df.iloc[:,:-1] #alle unabhängige var
y=df.iloc[:,-1] #nur die outcome werte

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)


In [ ]:
from sklearn.dummy import DummyClassifier
dummy_clf = DummyClassifier(strategy="most_frequent")
dummy_clf.fit(X, y)

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
accuracy_score(y, dummy_clf.predict(X))

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, dummy_clf.predict(X_test))

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn =KNeighborsClassifier()
knn.fit(X_train,y_train)
result =knn.predict(X_test)
accuracy_score(y_test, result)

In [ ]:
for i in [1,2,3,4,5,10,15,20,25,30,35]:
  print(i)
  knn =KNeighborsClassifier(n_neighbors=i)
  knn.fit(X_train,y_train)
  print("acc test " +str(accuracy_score(y_test, knn.predict(X_test))))
  print("acc train " + str(accuracy_score(y_train, knn.predict(X_train))))

In [ ]:
knn =KNeighborsClassifier(n_neighbors=1)

In [ ]:
# weather ex

In [ ]:
len(df['Location'].unique())
df["TRange"]= df["MaxTemp"] - df["MinTemp"]
df["MeanWind"] = (df["WindSpeed3pm"] + df["WindSpeed9am"])/2
#df["WindNormalized"] = df["WindGustSpeed"]/df["MeanWind"]

In [ ]:
bool_yes = {'Yes':1, "No":0}
df["RainToday"]= df["RainToday"].map(bool_yes)
df["RainTomorrow"]= df["RainTomorrow"].map(bool_yes)

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn import preprocessing

In [ ]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=100)
model.fit(X_train,y_train)
accuracy_score(y_test, model.predict(X_test))


In [ ]:
dummyClass = DummyClassifier(strategy="stratified")
dummyClass.fit(X_train,y_train)
accuracy_score(y_test, dummyClass.predict(X_test))

In [ ]:
# normalizer

In [ ]:
min_max_scaler = preprocessing.MinMaxScaler()
X_min_max_scaled= min_max_scaler.transform(X)
normalizer = preprocessing.Normalizer()
normalizer.fit(X_min_max_scaled)
X_normalized= normalizer.transform(X_min_max_scaled)
X_train_normalized, X_test_normalized, y_train_normalized, y_test_normalized = train_test_split(X_normalized, y, test_size=0.3)
model_scaled = RandomForestClassifier(n_estimators=100)
model_scaled.fit(X_train_normalized,y_train_normalized)
prediction = model_scaled.predict(X_test_normalized)
accuracy_score(y_test_normalized,prediction)

In [ ]:
X_train = X.drop(["Cloud3pm","Location"], axis=1)
X = X[X["Cloud3pm"]!=-1]

In [ ]:
# pipeline

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import FeatureUnion

In [ ]:
preprocessing_transformer = Pipeline(steps=[('scale_01', preprocessing.MinMaxScaler()), 
                                            ('normalizer', preprocessing.Normalizer())])

In [ ]:
model_pipeline = RandomForestClassifier(n_estimators=100)
my_pipeline = Pipeline(steps=[('preprocessing_transformer', preprocessing_transformer),
                              ('model_pipeline', model_pipeline)
                             ], verbose = True)
#test train, fit, predict, accuracy

In [ ]:
(df["MaxTemp"].values - df["MinTemp"].values).ndim

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import FeatureUnion
def sub_12(X):
    return np.reshape((X[:,0]-X[:,1]), (-1, 1))

def mean_12(X):
    return np.reshape(((X[:,0]+X[:,1])/2), (-1, 1))


def identity_input(X):
    return X

preprocessor_extension = ColumnTransformer(
    transformers=[
        ('Trange', FunctionTransformer(sub_12, validate=True), ['MaxTemp','MinTemp']),
        ('DiffHum', FunctionTransformer(sub_12, validate=True), ['Humidity3pm','Humidity9am']  ),
        ('MeanWind', FunctionTransformer(mean_12, validate=True), ['WindSpeed3pm','WindSpeed9am']  ),
        ('all', FunctionTransformer(identity_input, validate=True), [cname for cname in X.columns]  )
    ])

In [ ]:
verify=preprocessor_extension.fit_transform(X)

In [ ]:
preprocessor_extension = ColumnTransformer(
    transformers=[
        ('Trange', FunctionTransformer(sub_12, validate=True), ['MaxTemp','MinTemp']),
        ('DiffHum', FunctionTransformer(sub_12, validate=True), ['Humidity3pm','Humidity9am']  ),
        ('MeanWind', FunctionTransformer(mean_12, validate=True), ['WindSpeed3pm','WindSpeed9am']  ),
        #('all', FunctionTransformer(identity_input, validate=True), [cname for cname in X.columns]  )
    ], remainder ='passthrough')

verify=preprocessor_extension.fit_transform(X)
verify.shape

In [ ]:
from sklearn.model_selection import GridSearchCV

parameters = {
    'model_pipeline__n_estimators': [10,100,250]
    }
gs_clf = GridSearchCV(my_pipeline, parameters, cv=5, iid=False, n_jobs=-1)

gs_clf.fit(X_train, y_train)

gs_clf.best_params_
my_pipeline.set_params(**gs_clf.best_params_)

my_pipeline.fit(X_train,y_train)
preds = my_pipeline.predict(X_test)

# Evaluate the model
score = accuracy_score(y_test, preds)
print('Accuracy Score:', score)

In [ ]:
## mushroom

In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
for i in mushroom.columns[1:]:
    le.fit(mushroom.iloc[:,i])
    mushroom.iloc[:,i]= le.transform(mushroom.iloc[:,i])

In [ ]:
# ordinal encoder
oe = preprocessing.OrdinalEncoder()
oe.fit_transform(mushroom)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import cross_val_score

models = []
models.append(('LR', LogisticRegression()))
models.append(('KNN', KNeighborsClassifier()))
models.append(('NB', GaussianNB()))

# evaluate each model in turn
results = []
names = []
scoring = 'accuracy'
for name, model in models:

    cv_results = cross_val_score(model, X, y, cv=5, scoring=scoring)
    results.append(cv_results)
    names.append(name)
    msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
    print(msg)
# boxplot algorithm comparison
fig = plt.figure()
fig.suptitle('Algorithm Comparison')
ax = fig.add_subplot(111)
plt.boxplot(results)
ax.set_xticklabels(names)
plt.show()

In [ ]:
# evaluate each model in turn
results = []
names = []
scoring = 'accuracy'
for name, model in models:

    cv_results = cross_val_score(model, mushroom_color , y, cv=5, scoring=scoring)
    results.append(cv_results)
    names.append(name)
    msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
    print(msg)
# boxplot algorithm comparison
fig = plt.figure()
fig.suptitle('Algorithm Comparison')
ax = fig.add_subplot(111)
plt.boxplot(results)
ax.set_xticklabels(names)
plt.show()

In [ ]:
mushroom_color= pd.get_dummies(mushroom_color,  prefix_sep='_', columns=None)

In [ ]:
# evaluate each model in turn
results = []
names = []
scoring = 'accuracy'
for name, model in models:

    cv_results = cross_val_score(model, mushroom_color , y, cv=5, scoring=scoring)
    results.append(cv_results)
    names.append(name)
    msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
    print(msg)
# boxplot algorithm comparison
fig = plt.figure()
fig.suptitle('Algorithm Comparison')
ax = fig.add_subplot(111)
plt.boxplot(results)
ax.set_xticklabels(names)
plt.show()

In [ ]:
#one hot encoding
ohe = preprocessing.OneHotEncoder( sparse=False)

In [ ]:
# linear regression
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
# fit predict accuracy

In [ ]:
## naive bayes
# https://scikit-learn.org/stable/modules/naive_bayes.html
>>> from sklearn.datasets import load_iris
>>> from sklearn.model_selection import train_test_split
>>> from sklearn.naive_bayes import GaussianNB
>>> X, y = load_iris(return_X_y=True)
>>> X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=0)
>>> gnb = GaussianNB()
>>> y_pred = gnb.fit(X_train, y_train).predict(X_test)
>>> print("Number of mislabeled points out of a total %d points : %d"
...       % (X_test.shape[0], (y_test != y_pred).sum()))
Number of mislabeled points out of a total 75 points : 4

In [ ]:
## support vector machines svm
# https://scikit-learn.org/stable/modules/svm.html
>>> from sklearn import svm
>>> X = [[0, 0], [1, 1]]
>>> y = [0, 1]
>>> clf = svm.SVC()
>>> clf.fit(X, y)
SVC()
>>> clf.predict([[2., 2.]])
>>> clf.support_vectors_
>>> # get indices of support vectors
>>> clf.support_
>>> # get number of support vectors for each class
>>> clf.n_support_

## multi class classification
>>> X = [[0], [1], [2], [3]]
>>> Y = [0, 1, 2, 3]
>>> clf = svm.SVC(decision_function_shape='ovo')
>>> clf.fit(X, Y)
SVC(decision_function_shape='ovo')
>>> dec = clf.decision_function([[1]])
>>> dec.shape[1] # 4 classes: 4*3/2 = 6
6
>>> clf.decision_function_shape = "ovr"
>>> dec = clf.decision_function([[1]])
>>> dec.shape[1] # 4 classes

# docu for scaling and different kernels

In [ ]:
## visualization
import matplotlib.pyplot as plt

plt.close("all")
.plot
.plot.bar() - .diff().bar()
.plot.hist() - .diff().hist()
.plot.box()
.boxplot()
.plot.scatter()
.plot.kde()